## Normalization factor in HOTRG algorithm
In this notebook , we want to calculate the normalization of transfer matrix based on HOTRG algorithm.

In [2]:
import numpy as np
from math import pi
from scipy import linalg
import time, itertools
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

def SAVE_dATA( Cdata,RG_step,a_list,Dcut,name):

    dataframe = pd.DataFrame( Cdata )
    dataframe.index=['RG steps={}'.format(num_steps) for num_steps in range(RG_step) ]
    dataframe.columns = [ '%.5f'%a1  for a1 in a_list  ]

    dataframe = dataframe.stack()
    dataframe = dataframe.unstack(0)
    dataframe.index.name="Dcut="+str(Dcut)
    dataframe.to_csv(name )

def makeFolder(Filename,sep='/'):
    '''file name may contains several folders, so need to check
    individually'''
    subFolderPath = Filename.split('/')
    complete = '.'
    for subFolderP in subFolderPath:
        complete = "/".join([complete,subFolderP])
        dir_path = Path(complete)
        if not dir_path.exists():
            dir_path.mkdir()
    
def transfer_matrix(T):
    dims = T.shape
    result = np.zeros((dims[0],dims[2]))

    for i in range(dims[0]):
        for j in range(dims[2]):
            result[i][j] = np.trace(T[i,:,j,:])
    return result

    
def transfer_matrix_1x1(T):
    '''contract transfer matrix with unit cell 1x1x1'''
    dim = T.shape
    result = np.zeros((dim[0],dim[3]))
    for i in range(dim[0]):
        for j in range(dim[3]):
            for k in range(dim[1]):
                for l in range(dim[2]):
                    result[i][j] += T[i,k,l,j,k,l]
    return result



import numpy as np
from math import pi
from scipy import linalg



#==================================================================#
def DetT (si,sj,To,Ti,Pi):

    if si ==0 and sj == 0:
        T0 =Ti; T1=To; T2=To; T3=To
    elif si ==1 and sj == 0:
        T0 =Ti; T1=Pi; T2=To; T3=To
    elif si ==0 and sj == 1:
        T0 =Ti; T1=To; T2=To; T3=Pi
    elif si ==1 and sj == 1:
        T0 =Ti; T1=To; T2=Pi; T3=To

    return T0,T1,T2,T3
#==================================================================#


#==================================================================#
def DetP (si,sj,To,Pi):

    if si ==0 and sj == 0:
        T0 =Pi; T1=To; T2=To; T3=To
    elif si ==1 and sj == 0:
        T0 =To; T1=Pi; T2=To; T3=To
    elif si ==0 and sj == 1:
        T0 =To; T1=To; T2=To; T3=Pi
    elif si ==1 and sj == 1:
        T0 =To; T1=To; T2=Pi; T3=To

    return T0,T1,T2,T3
#==================================================================#


#==================================================================#
def DetTLIST (si,sj,To,Ti,Pi):

    if si ==0 and sj == 0:
        T0 =Ti; T1=To; T2=To; T3=To
    elif si ==1 and sj == 0:
        T0 =Ti; T1=Pi; T2=To; T3=To
    elif si ==0 and sj == 1:
        T0 =Ti; T1=To; T2=To; T3=Pi
    elif si ==1 and sj == 1:
        T0 =Ti; T1=To; T2=Pi; T3=To

    return T0,T1,T2,T3
#==================================================================#


#==================================================================#
def eig_all (theta):
    try:
        Y, Z = linalg.eigh(theta)
    except linalg.LinAlgError:
        Y, Z = linalg.eig(theta)
    piv = np.argsort(Y)[::-1]
    Y = np.sqrt(np.abs(Y[piv]))
    Z = np.conj(Z[:,piv].T)
    return Y,Z
#==================================================================#


#==================================================================#
def merge_two(T1,T2,UU,UUT):

    D1 = UU.shape[0]; D2=int(np.sqrt(UU.shape[1]))
    UU = np.reshape(UU,(D1,D2,D2))
    UUT = np.reshape(UUT,(D2,D2,D1))

    Aup =  np.tensordot(T1,UU, axes=(2,1))
    Adown =  np.tensordot(T2,UUT, axes=(0,1))
    AO = np.tensordot(Aup,Adown,axes=([0,1,4],[3,2,1]) )
    AO = np.transpose(AO, (3,2,1,0))
    return AO
#==================================================================#




##
##          3             3
##     2--(TO)--0     2--(T1)--0
##          1              1
##
##          3             3
##     2--(T3)--0     2--(T2)--0
##          1             1



#==================================================================#
def updat_pure( T0,T3,Bond,dcut):

    if Bond=='x':
        T0 = np.transpose( T0,(1,0,3,2) )
        T3 = np.transpose( T3,(1,0,3,2) )

    dimT0 = T0.shape
    dimT3 = T3.shape

    Aup = np.tensordot( T0, T0.conjugate(), axes= ([2,3],[2,3]))
    Adown= np.tensordot( T3, T3.conjugate(), axes= ([2,1],[2,1]))
    AA =  np.tensordot( Aup, Adown, axes= ([1,3],[1,3]))
    AA =  np.reshape( np.transpose (AA,(0,2,1,3)),(dimT0[0]*dimT3[0],dimT0[0]*dimT3[0]))

    Yo,Zo = eig_all(AA)
    dc1 = np.min([len(Yo),dcut])
    # dc1 = np.min([np.sum(Yo>10.**(-10)),dcut])
    UU = Zo [0:dc1,:];  UUT =(Zo.T[:,0:dc1]).conj();

    AO = merge_two(T0,T3,UU,UUT); N1 = np.max(abs(AO));
    NewT = AO/N1

    if Bond=='x': NewT = np.transpose(NewT,(1,0,3,2))

    return NewT,UU,UUT,N1
#==================================================================#



#==================================================================#
def updat_impurity(T0,T3,Bond,dcut,UU,UUT,N1):

    if Bond=='x':
        T0 = np.transpose( T0,(1,0,3,2) )
        T3 = np.transpose( T3,(1,0,3,2) )

    AO = merge_two(T0,T3,UU,UUT);
    NewT = AO/N1

    if Bond=='x': NewT = np.transpose(NewT,(1,0,3,2))

    return NewT
#==================================================================#




#==================================================================#
def Ising_square(T,bias=10**-4):


        Tax = np.ones((2,2))
        taix = []

        DT = np.zeros((2,2,2,2))
        iDT = np.zeros((2,2,2,2))

        for ii in range(2):
            DT [ii,ii,ii,ii] = 1.0
            iDT [ii,ii,ii,ii] = -1.0
        DT[0,0,0,0] += bias
        iDT [0,0,0,0] = 1

        Tax [0,0] = np.exp((1./T))
        Tax [0,1] = np.exp((-1./T))
        Tax [1,0] = np.exp((-1./T))
        Tax [1,1] = np.exp((1./T))


        Ya, Za = np.linalg.eigh(Tax)
        taix.append(np.tensordot(Za, np.diag(Ya**0.5),axes=(1,0)) )
        taix.append(np.tensordot(np.diag(Ya**0.5), (Za.conj().T) ,axes=(1,0)))


        DT =  np.tensordot( DT,taix[0],axes=(0,0))
        DT =  np.tensordot( DT,taix[0],axes=(0,0))
        DT =  np.tensordot( DT,taix[1],axes=(0,1))
        DT =  np.tensordot( DT,taix[1],axes=(0,1))

#        print '================='

        iDT =  np.tensordot(iDT,taix[0],axes=(0,0))
        iDT =  np.tensordot(iDT,taix[0],axes=(0,0))
        iDT =  np.tensordot(iDT,taix[1],axes=(0,1))
        iDT =  np.tensordot(iDT,taix[1],axes=(0,1))

        return DT, iDT
#==================================================================#

#==================================================================#
def Ising_exact(N,beta,dpp):

    Ten= np.zeros((N,N,N,N), dtype=dpp)
    iTen= np.zeros((N,N,N,N), dtype=dpp)

    for ii in range(N**4):
        Lab = dNb(N,ii,4)
        if np.mod(np.sum(Lab),N)==0:
            N0 = 0
            for jj in range(len(Lab)):
                if Lab[jj]==0: N0+=1

            NL = 4-N0
            VAL =np.exp(-2*beta)*( (np.sqrt(np.exp(beta*2.)+N-1))**(N0)*\
                (np.sqrt(np.exp(beta*2.)-1))**(NL) )/N

            Ten[Lab[0],Lab[1],Lab[2],Lab[3]]= VAL

        if np.mod(np.sum(Lab),N)==1:
            N0 = 0
            for jj in range(len(Lab)):
                if Lab[jj]==0: N0+=1

            NL = 4-N0
            VAL =np.exp(-2*beta)*( (np.sqrt(np.exp(beta*2.)+N-1))**(N0)*\
                (np.sqrt(np.exp(beta*2.)-1))**(NL) )/N

            iTen[Lab[0],Lab[1],Lab[2],Lab[3]]= VAL

    return Ten, iTen
#==================================================================#

#==================================================================#
def dNb(number,n,L):
    bb=[ ]
    for i in range(L-1,-1,-1):
        ii=number**i
        x,y= divmod(n, ii);n=y
        bb.append( x )
    return np.array(bb,dtype=int)
#==================================================================#


def log(a,b):
    '''return log_a(b)'''
    return np.log(b)/np.log(a)

def put_normalization_back(log_lambda_primes,Normalizations): 
    '''put the normalization back to lambda' to get $\widetilde{\lambda}$ '''
    result = []
    lastN = np.log(Normalizations[0]) # which should be 0
    total_RG_steps = len(log_lambda_primes)
    for i,lambda_i in enumerate(log_lambda_primes): #all take log
#         result.append(lambda_i+lastN)
        result.append(lambda_i/2**i+lastN)
        lastN = 2*lastN + np.log(Normalizations[i+1])*(1/2**(i+1))
    return result



First we want to examine the code between 2d and 3d. <br>
If we make the additional two index trivial ( that is, dimension = 1) , then all the calculation should be reduced to 
2d case. This way we can check somehow the label is as expected. <br>
However, since the Ising model is highly symmetryed, so it can just be a necessary condition.

In [11]:
# original case for 1x1 transfer matrix
Tc = 2.0/np.log(1.0 + np.sqrt(2))
dcut = 16
T_list = [Tc]
answer = []
EigVnums = dcut
trgstep = 4
RG = 0
for i,T in enumerate(T_list):
    eigenvalues = np.zeros((trgstep,EigVnums))
    DT, IDT = Ising_square(T,bias=0)
    T0 = DT;  iT0 = IDT; N_list = []; ti=0; FE_list = np.zeros(trgstep)
    TM = transfer_matrix( T0 )
    Y, Z = linalg.eigh(TM)
    
    Y = np.array(Y)[::-1]
    Y = Y[:2]
    print(np.log(Y))

    T1,UU,UUT,N1 = updat_pure( T0,T0,'y',dcut)
    T1 *= N1
    T2,UU,UUT,N2 = updat_pure( T1,T1,'x',dcut)
    T2 *= N2                                      #
    T0 = T2
    TM = transfer_matrix( T0 )
    Y, Z = linalg.eigh(TM)
    
    Y = np.array(Y)[::-1]
    Y = Y[:2]
    print(np.log(Y)/2)
    RG += 1
    T1,UU,UUT,N1 = updat_pure( T0,T0,'y',dcut)
    T1 *= N1
    T2,UU,UUT,N2 = updat_pure( T1,T1,'x',dcut)
    T2 *= N2                                       #
    T0 = T2
    TM = transfer_matrix( T0 )
    Y, Z = linalg.eigh(TM)
    
    Y = np.array(Y)[::-1]
    Y = Y[:2]
    print(np.log(Y)/4)
    RG += 1
    T1,UU,UUT,N1 = updat_pure( T0,T0,'y',dcut)
    T1 *= N1
    T2,UU,UUT,N2 = updat_pure( T1,T1,'x',dcut)
    T2 *= N2                                        #
    T0 = T2
    TM = transfer_matrix( T0 )
    Y, Z = linalg.eigh(TM)
    
    Y = np.array(Y)[::-1]
    Y = Y[:2]
    print(np.log(Y)/8)
    RG += 1


[1.22794718 0.34657359]
[2.01010508 1.57452077]
[3.78713656 3.58462585]
[7.4705735  7.37176378]


In [ ]:
# original case for 1x1 transfer matrix
Tc = 2.0/np.log(1.0 + np.sqrt(2))
dcut = 60
T_list = [Tc]
answer = []
EigVnums = dcut
trgstep = 5
Ys = []
N_list = []
for i,T in enumerate(T_list):
    eigenvalues = np.zeros((trgstep,EigVnums))
    DT, IDT = Ising_square(T,bias=0)
    T0 = DT;  iT0 = IDT; N_list = [1]; ti=0; FE_list = np.zeros(trgstep)
    for RG in range(trgstep):
        TM = transfer_matrix( T0 )
        Y, Z = linalg.eigh(TM)
        if len(Y) < EigVnums:
            totalEigs = len(Y)
            Num0 = EigVnums - totalEigs
            remains0 = [ 0 for i in range(Num0)]
            Y = list(Y)
            eigenvalues[RG,:] = np.array(Y[::-1] + remains0)
        else:
            eigenvalues[RG,:] = Y[:-1*EigVnums-1:-1]
        Y = np.array(Y)[::-1]
        Y = Y[0]

        Y = np.log(Y) 
#         print('RG=',RG)
#         print(Y/2**RG)
        Ys.append(Y)
        T1,UU,UUT,N1 = updat_pure( T0,T0,'y',dcut)
        T1 *= N1
        T2,UU,UUT,N2 = updat_pure( T1,T1,'x',dcut)
        N_list.append(100)
        T0 = T2*N2/100

# print(Ys)
# print(N_list)
print(put_normalization_back(Ys,N_list))

In [21]:
#factor = 2
1.2279471772995152
3.3270629744095768
11.68281035358393
45.20861830613169

# factor = 3
1.2279471772995152
2.921597866301412
9.655484813043108
36.69385103586024


[1.22794718 0.34657359]
[4.02021015 3.14904154]
[15.14854626 14.33850338]
[59.76458802 58.97411025]

SyntaxError: invalid syntax (<ipython-input-21-08e651509137>, line 1)

In [51]:
x = 4.020210152/3.3270629744095768

print(log(2,x))

x2 = 15.14854626/ 11.68281035358393 
print(log(2,x2))

x3 = 59.76458802 / 45.20861830613169

print(log(2,x3))

0.27302174494134657
0.3747919883102406
0.4026930796947974


In [52]:
x = 4.020210152/2.921597866301412

print(log(3,x))

x2 = 15.14854626/ 9.655484813043108
print(log(3,x2))

x3 = 59.76458802 / 36.69385103586024

print(log(3,x3))

0.2905515440471975
0.4099521252995282
0.44401844280219127


In [54]:
x = 4.020210152/2.6339157938496314

print(log(4,x))

x2 = 15.14854626/8.217074450784203
print(log(4,x2))

x3 = 59.76458802 /30.652527514372842

print(log(4,x3))

0.30503084760117266
0.4412413038553204
0.48164243320452205


# update
### 2020/12/5
Fix the bug in **updat_pur** due to lack of additional permutation on x direction
### 2020/12/4 
I found the function **updat_pure** in HOTRG seems strange, the order ( default 'y' -> 'x' ) would affect contraction result <br>
Here is my test for this function :


In [27]:
def updat_pure( T0,T3,Bond,dcut):

    if Bond=='x':
        T0 = np.transpose( T0,(1,0,3,2) )
        T3 = np.transpose( T3,(1,0,3,2) )

    dimT0 = T0.shape
    dimT3 = T3.shape

    Aup = np.tensordot( T0, T0.conjugate(), axes= ([2,3],[2,3]))
    Adown= np.tensordot( T3, T3.conjugate(), axes= ([2,1],[2,1]))
    AA =  np.tensordot( Aup, Adown, axes= ([1,3],[1,3]))
    AA =  np.reshape( np.transpose (AA,(0,2,1,3)),(dimT0[0]*dimT3[0],dimT0[0]*dimT3[0]))

    Yo,Zo = eig_all(AA)
    dc1 = np.min([np.sum(Yo>0),dcut])
    # dc1 = np.min([np.sum(Yo>10.**(-10)),dcut])
    UU = Zo [0:dc1,:];  UUT =(Zo.T[:,0:dc1]).conj();

    AO = merge_two(T0,T3,UU,UUT); N1 = np.max(abs(AO));
    NewT = AO/N1
    
    if Bond=='x': NewT=np.transpose(NewT,(1,0,3,2))  # transpose will return a new tensor, it is not inplace method.

    return NewT,UU,UUT,N1


def merge_two(T1,T2,UU,UUT):

    D1 = UU.shape[0]; D2=int(np.sqrt(UU.shape[1]))
    UU = np.reshape(UU,(D1,D2,D2))
    UUT = np.reshape(UUT,(D2,D2,D1))

    Aup =  np.tensordot(T1,UU, axes=(2,1))
    Adown =  np.tensordot(T2,UUT, axes=(0,1))
    AO = np.tensordot(Aup,Adown,axes=([0,1,4],[3,2,1]) )
    AO = np.transpose(AO, (3,2,1,0))
    return AO

def eig_all (theta):
    try:
        Y, Z = linalg.eigh(theta)
    except linalg.linalg.LinAlgError:
        Y, Z = linalg.eig(theta)
    piv = np.argsort(Y)[::-1]
    Y = np.sqrt(np.abs(Y[piv]))
    Z = np.conj(Z[:,piv].T)
    return Y,Z

##
##          3             3
##     2--(TO)--0     2--(T1)--0
##          1              1
##
##          3             3
##     2--(T3)--0     2--(T2)--0
##          1             1


In [28]:
Tc = 2.0/np.log(1.0 + np.sqrt(2))
dcut = 16
T_list = [Tc]
answer = []
trgstep = 3
for T in T_list:
    DT, IDT = HOTRG_two.Ising_square(T,bias=0)
    T0 = DT;  iT0 = IDT;
    for rg in range(trgstep):
        TM = transfer_matrix( T0 )
        Y, Z = linalg.eigh(TM)
        ans = Y[::-1]
        print(ans)
        answer.append(ans)
        T1,UU,UUT,N1 = updat_pure( T0,T0,'y',dcut)
        print(T1.shape)
        T2,UU,UUT,N2 = updat_pure( T1,T1,'x',dcut)
        print(T2.shape)
        T0 = T2*N1*N2

[3.41421356 1.41421356]
(4, 2, 4, 2)
(4, 4, 4, 4)
[9.26142438 3.87555639 0.11408573 0.04774057]
(16, 4, 16, 4)
(16, 16, 16, 16)
[6.87154318e+01 3.05673035e+01 1.73325940e-01 1.57567607e-01
 1.57567607e-01 2.64881512e-02 4.63835727e-03 4.63835727e-03
 4.63835727e-03 4.63835727e-03 8.12225740e-04 1.36540490e-04
 1.36540490e-04 1.24126592e-04 7.03835659e-07 3.13093546e-07]
(16, 16, 16, 16)
(16, 16, 16, 16)


In [5]:
x = np.random.rand(2,5,3,1,4,6)
y = np.transpose(x,[3,0,2,4,1,5])
z = np.transpose(y,[1,4,2,0,3,5])
print(y.shape)
print(z.shape)

(1, 2, 3, 4, 5, 6)
(2, 5, 3, 1, 4, 6)


In [ ]:
x = np.random.rand(1,2,3,4,5,6)
y = np.transpose(x,[2,0,1,5,3,4])
print(y)
z = np.transpose()